In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import kagglehub
import shutil
import os
import zipfile
import requests
from bs4 import BeautifulSoup

In [ ]:
!pip install requests beautifulsoup4 PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [8]:
# read text file
def extract_text_from_plain_text(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

In [9]:
def extract_pdf_link_from_html(url):
    """
    Extract a PDF link from an HTML page.

    Args:
        url (str): The URL of the HTML page.

    Returns:
        str or None: The extracted PDF link, or None if no PDF link is found.
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find a PDF link in the HTML
    pdf_link = None
    for a_tag in soup.find_all('a', href=True):
        if ".pdf" in a_tag['href']:  # Check if the href contains a PDF file
            pdf_link = a_tag['href']
            if not pdf_link.startswith("http"):  # Handle relative URLs
                pdf_link = url + pdf_link
            break
    return pdf_link

def download_pdf(url, save_path):
    """
    Download a PDF file from a given URL.

    Args:
        url (str): The URL of the PDF file.
        save_path (str): The local file path to save the downloaded PDF.

    Returns:
        None
    """
    try:
        response = requests.get(url, stream=True)
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded: {save_path}")
    except Exception as e:
        print(f"Failed to download {url}: {e}")

def extract_text_from_pdf(pdf_path):
    """
    Extract text from a PDF file.

    Args:
        pdf_path (str): The path to the PDF file.

    Returns:
        str or None: The extracted text, or None if an error occurs.
    """
    try:
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return None

In [10]:
def split_content_by_pages(df, content_column):
    """
    Split the content of each row in the DataFrame by the pattern "=== Page {number} ==="
    and return a new DataFrame with split pages.

    Args:
        df (pd.DataFrame): The input DataFrame containing file paths and content.
        content_column (str): The column name containing the text content to be split.

    Returns:
        pd.DataFrame: A new DataFrame with columns:
                      - 'file_path': The original file path.
                      - 'page_number': The extracted page number.
                      - 'page_content': The content of the corresponding page.
    """
    new_data = []
    for idx, row in df.iterrows():
        # Split the content by the pattern "=== Page {number} ==="
        pages = row[content_column].split("=== Page ")
        for page in pages:
            if page.strip():  # Ignore empty or whitespace-only entries
                # Attempt to extract the page number and content
                if " ===\n" in page:
                    page_number, page_content = page.split(" ===\n", 1)
                else:
                    page_number, page_content = page.strip(), ""

                # Append the parsed data as a dictionary
                new_data.append({
                    "file_path": row["file_path"],
                    "page_number": page_number.strip(),
                    "page_content": page_content.strip()
                })

    # Convert the list of dictionaries to a new DataFrame
    return pd.DataFrame(new_data)

In [13]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/nlp/OpenCourseWare/DontWatchThis/psychology_texts_all.csv')
# Split the 'content' column into individual pages
result_df = split_content_by_pages(df, content_column="content")
result_df.head()

,file_path,page_number,page_content
0,/content/drive/MyDrive/nlp/OpenCourseWare/hist...,1,STS 007 Class Notes\nIndustrial Revolution\nWh...
1,/content/drive/MyDrive/nlp/OpenCourseWare/hist...,2,substructure? Culture too\nAlso concept of “sc...
2,/content/drive/MyDrive/nlp/OpenCourseWare/hist...,3,Concentric circles that become feedback loops\...
3,/content/drive/MyDrive/nlp/OpenCourseWare/hist...,4,used in Newcastle area – first industrial land...
4,/content/drive/MyDrive/nlp/OpenCourseWare/hist...,5,"Pandaemonium 119, 130\nOliver Evans high press..."
